In [ ]:
import sys
sys.path = ['/home/etsymbalov/.local/lib/python3.6/site-packages'] + sys.path
import torch
from torch.nn.functional import elu

In [ ]:
# !python -m pip install torch==1.5.0+cu92 torchvision==0.6.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html --user

In [ ]:
torch.__version__

In [ ]:
# !python -m pip install alpaca-ml --user --upgrade --force
# я и руками ставил

In [ ]:
from torch.nn.functional import leaky_relu

In [ ]:
import os
import pickle
import random
from pathlib import Path

import pandas as pd
# import torch
# from torch.nn.functional import elu
import numpy as np
import matplotlib.pyplot as plt

from alpaca.uncertainty_estimator.masks import build_masks, DEFAULT_MASKS
from alpaca.model.ensemble import MLPEnsemble
from alpaca.uncertainty_estimator import build_estimator
from alpaca.analysis.metrics import get_uq_metrics

plt.rcParams['figure.facecolor'] = 'white'

In [ ]:
SEED = 10 
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

torch.cuda.set_device(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
def construct_estimator(model, model_type, name):
    if model_type == 'mask': 
        mask = masks[name]
        msk = build_estimator(
            'mcdue_masked', model, nn_runs=config['nn_runs'], dropout_mask=mask,
            dropout_rate=config['dropout_uq'])
        msk.tol_level=1e-5
        return msk
    elif model_type == 'emask': 
        mask = emasks[name]
        msk = build_estimator(
            'emcdue_masked', model, nn_runs=config['nn_runs'], dropout_mask=mask,
            dropout_rate=config['dropout_uq'])
        msk.tol_level=1e-5
        return msk
    else:
        return build_estimator(name, model)


class Evaluator:    
    def __init__(self, x_test, y_test, y_scaler, tag='standard'):
        self.x_test = torch.DoubleTensor(x_test).cuda()
        self.y_test = y_test
        self.unscale = lambda y : y_scaler.inverse_transform(y) 
        self.tag = tag
        self.results = []

    def bench(self, model, name, model_type='mask'): 
        predictions = model(self.x_test).cpu().detach().numpy()
        
        errors = np.abs(predictions - self.y_test)
        
        scaled_errors = self.unscale(predictions) - self.unscale(self.y_test)
        rmse = np.sqrt(np.mean(np.square(scaled_errors)))

        estimator = construct_estimator(model, model_type, name)
        if model_type == 'emask':
            name = 'e_' + name
        
        for run in range(config['n_ue_runs']):
            estimations = estimator.estimate(self.x_test)
            acc, ndcg, ll = get_uq_metrics(estimations, errors, 
                                           config['acc_percentile'],
                                           bins = [80, 95, 99]
                                          )
            self.results.append([acc, ndcg, ll, rmse, name, self.tag])
            if hasattr(estimator, 'reset'):
                estimator.reset()


In [ ]:
# folder = Path('./experiments/exp5_smot')
folder = Path('data/regression_2')
files = [file for file in os.listdir(folder) if file.endswith('.pickle')]


In [ ]:
files


In [ ]:
# with open(folder / file, 'rb') as f:
#     dct = pickle.load(f)
# with open('./paper_upgrade/A_log_exp.log', 'w') as f:
#     f.write(f'{cnt} / {len(files)}')

In [ ]:
DEFAULT_MASKS2 = DEFAULT_MASKS + ['decorrelating_sc', 'dpp']#, 'k_dpp']

In [ ]:
DEFAULT_MASKS

In [ ]:
data = []
errs = []
for cnt, file in enumerate(files[:3]):
#     if cnt < 235:
#         continue
    try:
        with open(folder / file, 'rb') as f:
            dct = pickle.load(f)
        with open('./A_log_exp.log', 'w') as f:
            f.write(f'{cnt} / {len(files)}')
        print(file)
        config = dct['config']
        config['n_ue_runs'] = 1
        config['acc_percentile'] = .1
        state_dict = dct['state_dict']
        x_train, y_train, x_val, y_val, x_scaler, y_scaler = dct['data']

        model = MLPEnsemble(
            config['layers'], n_models=config['n_ens'], activation = elu,
            reduction='mean')
        model.load_state_dict(state_dict)

        standard_evaluator = Evaluator(x_val, y_val, y_scaler, 'standard')
        masks = build_masks(DEFAULT_MASKS2)
        emasks = []
        for i in range(config['n_ens']):
            msk = build_masks(DEFAULT_MASKS2)
            emasks.append(msk)
        emasks = {key: [e[key] for e in emasks] for key in masks.keys()}

        single_model = model.models[2]
        for name in masks: 
            print(name, end = '|')
            standard_evaluator.bench(single_model, name, 'mask')
        standard_evaluator.bench(model, 'eue', 'ensemble')    
        for name in emasks: 
            print(name, end = '*|')
            standard_evaluator.bench(model, name, 'emask')
        mask_df = pd.DataFrame(standard_evaluator.results, 
                           columns=['Acc', 'NDCG', 'LL',
                                    'RMSE', 'Mask', 'Tag'])
        mask_df['fname'] = file
        data.append(mask_df)
        pd.concat(data).to_csv('./AAAA_experiment_results.csv', index = None)
    except Exception as e:
        errs.append([e,cnt,file])

In [ ]:
pd.concat(data).head(50)